In [2]:
import gensim
import numpy as np 
import pandas as pd
import pickle
import csv

## Parsing raw reddit posts

In [24]:
import csv
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
# print(lemmatizer.lemmatize("cats"))

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')

def parse_reddit_csv(filename):
    print("Reading from", filename)
    csv_cols = []
    authors = {}
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            # Remove numbers, punctuation
            row['selftext'] = re.sub(r'\d+', '', row['selftext'])
            row['title'] = re.sub(r'\d+', '', row['title'])
            # Tokenize the post text (selftext) and post title
            post_tokens = tokenizer.tokenize(row['selftext'])
            title_tokens = tokenizer.tokenize(row['title'])
            # Filter out stopwords
            post_tokens = [w for w in post_tokens if not w in stop_words]
            title_tokens = [w for w in title_tokens if not w in stop_words]
            # Lemmatize the post text (reduce words to word stems i.e. cats->cat, liked->like)
            post_tokens = [lemmatizer.lemmatize(w, 'n') for w in post_tokens]
            post_tokens = [lemmatizer.lemmatize(w, 'v') for w in post_tokens]
            title_tokens = [lemmatizer.lemmatize(w, 'n') for w in title_tokens]
            title_tokens = [lemmatizer.lemmatize(w, 'v') for w in title_tokens]
            csv_cols.append({'author': row['author'],
                             'selftext': post_tokens,
                             'title': title_tokens,
                             'post_id': row['id']})
            # Add author mapping
            if row['author'] not in authors:
                authors[row['author']] = []
            authors[row['author']].append(row['id'])
    return csv_cols, authors

[nltk_data] Downloading package wordnet to /home/cephcyn/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cephcyn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
parsed, authors= parse_reddit_csv('data/final_proj_data_preprocessed_1000sample.csv')

with open('sample1000_parse.pickle', 'wb') as handle:
    pickle.dump(parsed, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('sample1000_parse_authors.pickle', 'wb') as handle:
    pickle.dump(authors, handle, protocol=pickle.HIGHEST_PROTOCOL)

Reading from data/final_proj_data_preprocessed_1000sample.csv


## Calculate post embeddings (Word2Vec)
(using selftext only)

The first (thereafter called W2VWeighted) is calculated by weighing the contribution of each word embedding by the inverse of its relative frequency to the final sentence embedding.

In doing so, the contributions of the most common words are minimized.

The second (thereafter called W2V-SIF) is calculated by first taking the weighed sentence embedding before removing the first principal component from it.

Sanjeev Arora, Yingyu Liang, and Tengyu Ma. 2017.\
A simple but tough-to-beat baseline for sentence embeddings. In ICLR.

In [ ]:
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('model/GoogleNews-vectors-negative300.bin', binary=True)  

In [ ]:
from collections import Counter

sen_emb = {}
for i in range(len(parsed)):
    counts = Counter(parsed[i]['selftext']).items()
    freq = pd.DataFrame(counts)
    freq = freq.rename(columns={0: "word", 1: 'freq'})
    # Weight by inverse relative frequency
    freq['inv_rfreq'] = freq['freq'].sum()/freq['freq']
    unknowns = []
    emb_dict = {}
    for w in freq['word'].to_list():
        try:
            emb = model[w]
            emb_dict.update({w:emb})
        except:
            unknowns.append(w)
    emb_value = pd.DataFrame(emb_dict).transpose().reset_index()
    emb_value = emb_value.rename(columns={'index': "word"})
    emb_value_list = emb_value.iloc[:, 1:301].mul(freq['inv_rfreq'], axis = 0).sum().to_list()
    sen_emb.update({parsed[i]['post_id']:emb_value_list})       

In [ ]:
with open('sample1000_embed_w2v.pickle', 'wb') as handle:
    pickle.dump(sen_emb, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Calculate post topics (LDA)
(using both selftext and title)

A Bag of Words (BoW) corpus was obtained before a term frequency-inverse document frequency (TF-IDF) corpus was derived from it.

Topic modeling was then performed on both the BoW corpus (thereafter LDA-BoW) and
TF-IDF corpus (thereafter LDA-TFIDF) with the number of topics set to 30, in line with the number of clusters used.

The document-topic mapping of each post is then used for computing cosine similarities with all other posts

In [ ]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary([parsed[i]['selftext'] for i in range(len(parsed))])
print(dictionary)

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(parsed[i]['selftext']) for i in range(len(parsed))]
# for doc in corpus:
#     print([[dictionary[id], np.around(freq, decimals=2)] for id, freq in doc])

# TF-IDF (term freq, inverse document freq) representation
from gensim import models
tfidf = models.TfidfModel(corpus)
# for doc in tfidf[corpus]:
#     print([[dictionary[id], np.around(freq, decimals=2)] for id, freq in doc])

In [ ]:
from gensim.models import LdaModel

def get_topics(corpus):
    # Train LDA model, get model & topic vectors
    # Set training parameters.
    num_topics = 30
    chunksize = 100
    passes = 20
    iterations = 400
    eval_every = 100  # None = Don't evaluate model perplexity, takes too much time.

    # Make a index to word dictionary.
    temp = dictionary[0]  # This is only to "load" the dictionary.
    id2word = dictionary.id2token

    model = LdaModel(
        corpus=corpus,
        id2word=id2word,
        chunksize=chunksize,
        alpha='auto',
        eta='auto',
        iterations=iterations,
        num_topics=num_topics,
        passes=passes,
        eval_every=eval_every
    )
    
    # Get basic evaluation
    top_topics = model.top_topics(corpus) #, num_words=20)

    # Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
    avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
    print('Average topic coherence: %.4f.' % avg_topic_coherence)
    
    # Get topic vectors
    all_topics = model.get_document_topics(corpus, per_word_topics=True)
    all_topics = [(doc_topics, word_topics, word_phis) for doc_topics, word_topics, word_phis in all_topics]
    sen_top = {}
    for i in range(len(parsed)):
        # These are in the same order as the documents themselves.
        doc_topics, word_topics, phi_values = all_topics[i]
        # Generate the topic VECTOR not just list of topics
        doc_topic_vector = [0] * num_topics
        for topic in doc_topics:
            doc_topic_vector[topic[0]] = topic[1]
        sen_top.update({parsed[i]['post_id']:doc_topic_vector})
    
    return model, sen_top

In [ ]:
# Get bow data
print("Generating topics for BOW...")
model_bow, sen_top_bow = get_topics(corpus)

# Get tfidf data
print("Generating topics for TFIDF...")
model_tfidf, sen_top_tfidf = get_topics(tfidf[corpus])

In [ ]:
# Save bow data
with open('sample1000_model_top_bow.pickle', 'wb') as handle:
    pickle.dump(model_bow, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('sample1000_embed_top_bow.pickle', 'wb') as handle:
    pickle.dump(sen_top_bow, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save tfidf data
with open('sample1000_model_top_tfidf.pickle', 'wb') as handle:
    pickle.dump(model_tfidf, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('sample1000_embed_top_tfidf.pickle', 'wb') as handle:
    pickle.dump(sen_top_tfidf, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Calculate Pairwise Cosine Similarity & Cluster

In [7]:
import pickle

with open('sample1000_embed_top_bow.pickle', 'rb') as handle:
    sen_emb = pickle.load(handle)

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

d = pd.DataFrame(sen_emb).transpose()
sim_mat = cosine_similarity(d)

In [9]:
post = d.index.to_list()
post_emb = dict(zip(post, sim_mat))

In [13]:
import random
import numpy as np
import pandas as pd

def Similarity_clustering(similarity_dict, m, n):
    clusters = {};
    unselected_posts = similarity_dict.copy()
    post_keys = list(unselected_posts.keys())
    unselected_keys = list(unselected_posts.keys())
    cluster_size = int(np.ceil(n / m))
    while len(unselected_posts) != 0:
        selected_post = random.choice(unselected_keys)
        # labeling the selected row
        emb_dict = dict(zip(post_keys, unselected_posts[selected_post]))
        # only sort the unselected columns
        sim = {k: emb_dict[k] for k in unselected_keys}
        sim_sort = [k for k in sorted(sim.items(), key=lambda item: item[1])][::-1]
        sim_most = sim_sort[0:cluster_size]
        clusters[selected_post] = sim_most
        # deleted the selected rows from the unselected
        for p in sim_most:
            del unselected_posts[p[0]]
        unselected_keys = list(unselected_posts.keys())
        cluster_size = int(np.floor(n / m))
    return clusters

In [21]:
numClusters = 6
numTotalPosts = 1000
cluster = Similarity_clustering(post_emb, numClusters, numTotalPosts)
print(cluster)

with open('sample1000_clust_top_tfidf.pickle', 'wb') as handle:
    pickle.dump(cluster, handle, protocol=pickle.HIGHEST_PROTOCOL)

{'91srwq': [('91srwq', 1.0), ('dk4fsz', 0.9999873771529322), ('dinmi1', 0.9999827741866195), ('at7vl6', 0.9999827264977285), ('7sb0d3', 0.9999705934474751), ('7rmlxn', 0.99996277785444), ('8gf8br', 0.9999575510335756), ('7qbpme', 0.9999453457591536), ('bkediu', 0.9999391977417359), ('d4y131', 0.999937172175664), ('cjysbh', 0.9999163864118301), ('b69acp', 0.9999143729303526), ('9f2zv1', 0.9999115334681732), ('bw5sjh', 0.9999086624459855), ('cy7zxe', 0.9999044457568479), ('9hwiu4', 0.9998954116404717), ('b4fvo0', 0.9998917121637438), ('cppqoq', 0.9998906309290124), ('d4gwp1', 0.9998857650522374), ('cy4rc8', 0.999875801494631), ('7l8e5l', 0.9998657569104734), ('8ur1t0', 0.9998643182295665), ('bqqyiw', 0.9998524565603104), ('8nkwkg', 0.9998480403006698), ('dk4q1x', 0.9998439290953011), ('cjx0ii', 0.9998396169385332), ('dao2fm', 0.999836989878144), ('c9ibxr', 0.9998358633894534), ('bnbjiv', 0.9998168547616552), ('9akkq6', 0.999807365644723), ('85wkbf', 0.9997922341888669), ('bdxz9y', 0.9997

In [23]:
# Transform clusters into a post_id:cluster_id dict
transformed_cluster = {}
clust_num = 0
for key in cluster.keys():
    for post in cluster[key]:
        transformed_cluster[post[0]] = clust_num
    clust_num += 1
print(transformed_cluster)
    
with open('sample1000_clustdict_top_tfidf.pickle', 'wb') as handle:
    pickle.dump(transformed_cluster, handle, protocol=pickle.HIGHEST_PROTOCOL)

{'91srwq': 0, 'dk4fsz': 0, 'dinmi1': 0, 'at7vl6': 0, '7sb0d3': 0, '7rmlxn': 0, '8gf8br': 0, '7qbpme': 0, 'bkediu': 0, 'd4y131': 0, 'cjysbh': 0, 'b69acp': 0, '9f2zv1': 0, 'bw5sjh': 0, 'cy7zxe': 0, '9hwiu4': 0, 'b4fvo0': 0, 'cppqoq': 0, 'd4gwp1': 0, 'cy4rc8': 0, '7l8e5l': 0, '8ur1t0': 0, 'bqqyiw': 0, '8nkwkg': 0, 'dk4q1x': 0, 'cjx0ii': 0, 'dao2fm': 0, 'c9ibxr': 0, 'bnbjiv': 0, '9akkq6': 0, '85wkbf': 0, 'bdxz9y': 0, '8yjjmj': 0, 'dmp3rd': 0, 'beeham': 0, 'd182gu': 0, '8bnhwt': 0, 'cumrtb': 0, 'ca5943': 0, 'b2m2zv': 0, 'd4nyln': 0, '95rzdi': 0, '7qlequ': 0, 'chooph': 0, 'doohfs': 0, 'd2jcw1': 0, 'butpqq': 0, 'cluq3w': 0, '8cmyxx': 0, '84n43b': 0, '7qyrw0': 0, '7rllhr': 0, 'cichas': 0, 'd0p7b4': 0, 'dj6cyi': 0, 'bgsneo': 0, '7pf5we': 0, 'czesor': 0, 'bka12a': 0, 'cm0tj4': 0, '7lp4wa': 0, 'dgqjhx': 0, 'b3rzek': 0, '8es5oz': 0, 'd1ns76': 0, 'c7fet6': 0, '85w9re': 0, '8hoerr': 0, 'dpf3a2': 0, '8418lv': 0, '9e9ist': 0, '7k214l': 0, 'ckajnl': 0, 'dfwde8': 0, '9hs2ol': 0, '9dzn72': 0, 'asat9m': 0

## Score Clusters: Calculate Same-Author-Score

In [40]:
import pickle

# Read clusters
with open('sample1000_clust_top_tfidf.pickle', 'rb') as handle:
    clusters = pickle.load(handle)
with open('sample1000_clustdict_top_tfidf.pickle', 'rb') as handle:
    clustdict = pickle.load(handle)
# Read author list
with open('sample1000_parse_authors.pickle', 'rb') as handle:
    authors = pickle.load(handle)

In [42]:
import itertools

num_clust_pair = 0
num_total_pair = 0

for auth in authors:
    if len(authors[auth]) < 2:
        continue
    for pair in itertools.product(authors[auth],authors[auth]):
        if pair[0] == pair[1]:
            continue
        num_total_pair += 1
        if clustdict[pair[0]] == clustdict[pair[1]]:
            num_clust_pair += 1

score_sas = (num_clust_pair / num_total_pair) - (1/len(clusters))
print(score_sas)

0.7142857142857142


## Score Clusters: Calculate Jaccard Score